In [1]:
import os
#import pandas as pd 
import modin.pandas as pd # emails went to 236
from tqdm import tqdm
import datetime
from p_tqdm import p_map
from functools import partial
import gc

In [2]:
time_resolution = 30
time_window = 10
lk_path = '/mnt/data0/lkyin/'
c_path = './data_smoothed'+str(time_resolution+time_window)+'/commits/'
e_path = './data_smoothed'+str(time_resolution+time_window)+'/emails/'
mix_path = './data_smoothed'+str(time_resolution+time_window)+'/mix/'

In [3]:
print('reading commits...')
df_data = pd.read_csv(lk_path+'commits_final.csv')
print('grouping by project...')
df = dict(tuple(df_data.groupby(df_data['project_name'])))
#to_path = './weekly_data/commits/'
#if not os.path.exists(to_path):
#	os.makedirs(to_path)


reading commits...


To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()



grouping by project...


In [4]:
df_data.columns

Index(['project_name', 'list_name', 'date', 'month', 'message_id',
       'sender_name', 'sender_email', 'author_name', 'author_email',
       'file_name', 'loc', 'ref_or_sha', 'subject', 'commit_type',
       'author_full_name', 'is_bot', 'is_coding', 'dealised_author_full_name'],
      dtype='object')

In [5]:
df.keys()

dict_keys(['abdera', 'accumulo', 'ace', 'activemq', 'adffaces', 'agila', 'airavata', 'airflow', 'allura', 'alois', 'amaterasu', 'ambari', 'ant', 'any23', 'apex', 'apisix', 'ariatosca', 'aries', 'asterixdb', 'atlas', 'aurora', 'awf', 'batchee', 'beam', 'beehive', 'bigtop', 'bloodhound', 'bluesky', 'blur', 'bookkeeper', 'brooklyn', 'buildr', 'bval', 'calcite', 'carbondata', 'cassandra', 'cayenne', 'celix', 'chemistry', 'chukwa', 'clerezza', 'click', 'climate', 'cloudstack', 'cmda', 'commons', 'commonsrdf', 'composer', 'concerted', 'cordova', 'corinthia', 'couchdb', 'crunch', 'ctakes', 'curator', 'cxf', 'daffodil', 'datafu', 'datasketches', 'deltacloud', 'deltaspike', 'derby', 'devicemap', 'directmemory', 'directory', 'dolphinscheduler', 'drill', 'droids', 'druid', 'dubbo', 'eagle', 'echarts', 'edgent', 'empire', 'esme', 'etch', 'falcon', 'felix', 'fineract', 'flex', 'flink', 'flume', 'fluo', 'freemarker', 'ftpserver', 'gearpump', 'geode', 'giraph', 'gobblin', 'gora', 'gossip', 'graffito'

In [6]:
df_data['datetime'] = df_data['date'].apply(lambda x: pd.Period(x))
df_data['year'] = df_data['datetime'].apply(lambda x: x.year)
#df_data['dayofyear'] = df_data['datetime'].apply(lambda x: x.dayofyear)
#df_data['weekofyear'] = df_data['datetime'].apply(lambda x: x.weekofyear)
df_data['week'] = df_data['datetime'].apply(lambda x: pd.Period(x, freq='W'))
df_data['day'] = df_data['datetime'].apply(lambda x: pd.Period(x, freq='D'))

In [7]:
df_data.head(10)

,project_name,list_name,date,month,message_id,sender_name,sender_email,author_name,author_email,file_name,...,subject,commit_type,author_full_name,is_bot,is_coding,dealised_author_full_name,datetime,year,week,day
0,etch,commits,2009-01-20 23:25:35,4,<20090120232535.BC6AC2388892@eris.apache.org>,sccomer,sccomer@apache.org,sccomer,NaN,incubator/etch/trunk/binding-csharp/runtime/sr...,...,svn commit: r736169 - in /incubator/etch/trunk...,svn,robert scott comer,False,True,Robert Scott Comer,2009-01-20 23:25:35,2009,2009-01-19/2009-01-25,2009-01-20
1,etch,commits,2009-01-20 23:25:35,4,<20090120232535.BC6AC2388892@eris.apache.org>,sccomer,sccomer@apache.org,sccomer,NaN,incubator/etch/trunk/binding-java/runtime/src/...,...,svn commit: r736169 - in /incubator/etch/trunk...,svn,robert scott comer,False,True,Robert Scott Comer,2009-01-20 23:25:35,2009,2009-01-19/2009-01-25,2009-01-20
2,etch,commits,2009-01-22 17:43:20,4,<20090122174320.717AC23888E6@eris.apache.org>,sccomer,sccomer@apache.org,sccomer,NaN,incubator/etch/trunk/antsetup.bat,...,svn commit: r736725 - /incubator/etch/trunk/an...,svn,robert scott comer,False,True,Robert Scott Comer,2009-01-22 17:43:20,2009,2009-01-19/2009-01-25,2009-01-22
3,etch,commits,2009-01-26 17:55:13,4,<20090126175513.C548823889C1@eris.apache.org>,sccomer,sccomer@apache.org,sccomer,NaN,incubator/etch/trunk/binding-csharp/compiler/s...,...,svn commit: r737767 - in /incubator/etch/trunk...,svn,robert scott comer,False,True,Robert Scott Comer,2009-01-26 17:55:13,2009,2009-01-26/2009-02-01,2009-01-26
4,etch,commits,2009-01-26 17:55:13,4,<20090126175513.C548823889C1@eris.apache.org>,sccomer,sccomer@apache.org,sccomer,NaN,incubator/etch/trunk/binding-java/compiler/src...,...,svn commit: r737767 - in /incubator/etch/trunk...,svn,robert scott comer,False,True,Robert Scott Comer,2009-01-26 17:55:13,2009,2009-01-26/2009-02-01,2009-01-26
5,etch,commits,2009-01-26 17:55:13,4,<20090126175513.C548823889C1@eris.apache.org>,sccomer,sccomer@apache.org,sccomer,NaN,incubator/etch/trunk/binding-xml/compiler/src/...,...,svn commit: r737767 - in /incubator/etch/trunk...,svn,robert scott comer,False,True,Robert Scott Comer,2009-01-26 17:55:13,2009,2009-01-26/2009-02-01,2009-01-26
6,etch,commits,2009-01-26 17:55:13,4,<20090126175513.C548823889C1@eris.apache.org>,sccomer,sccomer@apache.org,sccomer,NaN,incubator/etch/trunk/compiler/src/main/java/et...,...,svn commit: r737767 - in /incubator/etch/trunk...,svn,robert scott comer,False,True,Robert Scott Comer,2009-01-26 17:55:13,2009,2009-01-26/2009-02-01,2009-01-26
7,etch,commits,2009-01-26 17:55:13,4,<20090126175513.C548823889C1@eris.apache.org>,sccomer,sccomer@apache.org,sccomer,NaN,incubator/etch/trunk/compiler/src/main/java/et...,...,svn commit: r737767 - in /incubator/etch/trunk...,svn,robert scott comer,False,True,Robert Scott Comer,2009-01-26 17:55:13,2009,2009-01-26/2009-02-01,2009-01-26
8,etch,commits,2009-01-26 17:59:25,4,<20090126175925.3EE0323888E6@eris.apache.org>,sccomer,sccomer@apache.org,sccomer,NaN,incubator/etch/trunk/installers/etch-maven-ins...,...,svn commit: r737770 - /incubator/etch/trunk/in...,svn,robert scott comer,False,True,Robert Scott Comer,2009-01-26 17:59:25,2009,2009-01-26/2009-02-01,2009-01-26
9,etch,commits,2009-01-27 17:30:44,4,<20090127173044.7C8FD23889B7@eris.apache.org>,sccomer,sccomer@apache.org,sccomer,NaN,incubator/etch/trunk/binding-csharp/runtime/sr...,...,svn commit: r738160 - in /incubator/etch/trunk...,svn,robert scott comer,False,True,Robert Scott Comer,2009-01-27 17:30:44,2009,2009-01-26/2009-02-01,2009-01-27


In [8]:
df_data['day'].describe()

count        8604294
unique          5963
top       2015-01-27
freq          351139
Name: day, dtype: object

In [9]:
df_data['day'][10]-df_data['day'][0]

<7 * Days>

In [16]:
df_data['day'][10]-df_data['day'][0] == 7* pd.Timedelta("1 day")

False

In [3]:
print('reading commits...')
df_data = pd.read_csv(lk_path+'commits_final.csv')
df_data = df_data.loc[df_data.date.str.len()==19]
df_data['day'] = df_data['date'].apply(lambda x: pd.Period(x, freq='D'))
df_data.to_csv("./temp_data.csv", index=False)
print(df_data.dtypes)

reading commits...


To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()



project_name                    object
list_name                       object
date                            object
month                            int64
message_id                      object
sender_name                     object
sender_email                    object
author_name                     object
author_email                    object
file_name                       object
loc                            float64
ref_or_sha                      object
subject                         object
commit_type                     object
author_full_name                object
is_bot                            bool
is_coding                         bool
dealised_author_full_name       object
day                          period[D]
dtype: object


In [3]:
# Use normal pandas here instead
import pandas as pd
df_data = pd.read_csv("./temp_data.csv")
#df_data['day'] = df_data['day'].apply(lambda x: pd.Period(x))
#df_data['day'] = df_data['day'].astype(str)
print('grouping by project...')
df = dict(tuple(df_data.groupby(df_data['project_name'])))
to_path = c_path
if not os.path.exists(to_path):
	os.makedirs(to_path)

grouping by project...


In [4]:
def period_save(df, project):
	daily_df_dict = dict(tuple(df[project].groupby(df[project]['day'])))
	dayint = 0
	for day0, value in daily_df_dict.items():
		day0 = pd.Period(day0)
		#print(day0)
		break
	periodly_df = pd.DataFrame()
	periodnum = 0

	for today in daily_df_dict:		
		daily_df = daily_df_dict[today]
		daily_df = daily_df[daily_df['dealised_author_full_name'].notna()]
		if daily_df.empty: continue
		daily_df['day'] = daily_df['day'].apply(lambda x: pd.Period(x))
		while pd.Period(today) > (day0 + (dayint*pd.Timedelta("1 day"))):
			dayint+=1
		
		if dayint > time_resolution+time_window:
			periodnum+=1
			periodstr = str(periodnum).zfill(3)
			file_path = to_path + '{}__{}.csv'.format(project, periodstr)
			#print("Saving: "+ file_path)
			periodly_df.to_csv(file_path, index=False)
			periodly_df = pd.DataFrame()
			day0+= (time_resolution+time_window)*pd.Timedelta("1 day")
			dayint-= (time_resolution+time_window)

		periodly_df = pd.concat([periodly_df, daily_df], ignore_index=True)
	
	gc.collect()



In [5]:
print('grouping by period...')

for project in tqdm(df):
	period_save(df, project)
print('Commits Done.')

grouping by period...


100%|██████████| 263/263 [21:57<00:00,  5.01s/it]   

Commits Done.


In [6]:
import modin.pandas as pd
print('reading emails...')
df_data = pd.read_csv(lk_path+'emails_final.csv')
df_data = df_data.loc[df_data.date.str.len()==19]
df_data['day'] = df_data['date'].apply(lambda x: pd.Period(x, freq='D'))
df_data.to_csv("./temp_data.csv", index=False)
print(df_data.dtypes)


To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()



reading emails...


Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.


project_name                    object
list_name                       object
date                            object
month                          float64
message_id                      object
sender_name                     object
references                      object
sender_email                    object
in_reply_to                     object
cc_list                         object
receiver_email                  object
subject                         object
body                            object
from_commit                     object
author_full_name                object
is_bot                            bool
dealised_author_full_name       object
day                          period[D]
dtype: object


In [7]:
import pandas as pd
df_data = pd.read_csv("./temp_data.csv")
print('grouping by project...')
df = dict(tuple(df_data.groupby(df_data['project_name'])))
to_path = e_path
if not os.path.exists(to_path):
	os.makedirs(to_path)
gc.collect()

grouping by project...


6

In [8]:
print('grouping by period...')

for project in tqdm(df):
	period_save(df, project)
print('Commits Done.')

grouping by period...


100%|██████████| 274/274 [11:41<00:00,  2.56s/it]

Commits Done.


In [9]:
gc.collect()

0